In [3]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import math
import os
# import natsort
import torchvision
from PIL import Image
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torch.utils.data as data

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
EPOCHS = 20
BATCH_SIZE = 25
LEARNING_RATE = 0.003
IMG_SIZE = 64
CONV_SIZE = math.floor((((IMG_SIZE-2)/2)-2)/2)
INPUT_SIZE = (IMG_SIZE**2)*BATCH_SIZE
LOG_INTERVAL = 10

TRAIN_DATA_PATH = r"C:\Users\mdro\Desktop\programming\datasets\images\lungs_kaggle\train\normal"
TEST_DATA_PATH = r"C:\Users\mdro\Desktop\programming\datasets\images\lungs_kaggle\valid\normal"

TRANSFORM_IMG = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor()])

In [7]:
class CustomDataset(data.Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        print(image)
        tensor_image = self.transform(image)
        return tensor_image

train_data = CustomDataset(TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0)

test_data = CustomDataset(TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

print('  Train len:',len(train_data))
print('   Test len:',len(test_data))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\mdro\\Desktop\\programming\\datasets\\images\\lungs_kaggle\\train\\normal'

In [ ]:
size5 = 256
size4 = 128
size3 = 64
size2 = 32
size1 = 16

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(INPUT_SIZE, size5)
        self.fc2 = nn.Linear(size5, size4)
        self.fc3 = nn.Linear(size4, size3)
        self.fc4 = nn.Linear(size3, size2)
        
        self.fc51 = nn.Linear(size2, size1)
        self.fc52 = nn.Linear(size2, size1)
        self.fc6 = nn.Linear(size1, size2)
        
        self.fc7 = nn.Linear(size2, size3)
        self.fc8 = nn.Linear(size3, size4)
        self.fc9 = nn.Linear(size4, size5)
        self.fc10 = nn.Linear(size5, INPUT_SIZE)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return self.fc51(x), self.fc52(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        z = F.relu(self.fc6(z))
        z = F.relu(self.fc7(z))
        z = F.relu(self.fc8(z))
        z = F.relu(self.fc9(z))
        return torch.sigmoid(self.fc10(z))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, INPUT_SIZE))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, INPUT_SIZE), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

train_losses = []
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device) 
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    train_loss_ = train_loss / len(train_loader.dataset)       
    train_losses.append(train_loss_)
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss_))

test_losses = []
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:       
                n = min(data.size(0), 5)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(BATCH_SIZE, 1, IMG_SIZE, IMG_SIZE)[:n]])
                save_image(comparison.cpu(),
                         'reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('====> Test set loss: {:.4f}'.format(test_loss))


if __name__ == "__main__":
    for epoch in range(1, EPOCHS + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(5, size1).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(BATCH_SIZE*5, 1, IMG_SIZE, IMG_SIZE),
                       'sample_' + str(epoch) + '.png')
    plt.plot(train_losses, 'b', test_losses, 'orange')